<div class="alert alert-info">
           <h1 style="font-size:25px;"><strong>Projet 5</strong></h1>
           <h1 style="font-size:25px;"><strong>Segmentez des clients d'un site e-commerce</strong></h1>
</div>

<div class="alert alert-info">
           <h1 style="font-size:25px;"><strong>Notebook Simulation</strong></h1>
</div>

## <font color="#00afe6">Sommaire</font>

[Imports, Chargemernts et connaisance de données. ](#section_A)                                 
[01. Ensemble du jeu de données](#section_1)               
[02. Simulation pour déterminer la fréquence nécessaire de mise à jour](#section_2)                    
[03. Modèle 1](#section_3)               
[04. Modèle 2](#section_4)              
[05. Simulation](#section_5)                                                                               
[06. Conclusion](#section_6)                                    

<div class="alert alert-info">
    <font id="section_A">
        <h1 style="font-size:25px;"><strong>Imports, Configuration, Versions, Chargements et connaisance de données. </strong></h1>
</div>

In [1]:
#imports
import numpy as np 
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
import seaborn as sns
import sys
import IPython as ip
import statsmodels as stat
import sklearn

from matplotlib.patches import Circle
import folium
import pickle
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import silhouette_score

import datetime
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from sklearn.preprocessing import OneHotEncoder, StandardScaler


import warnings

# Suprimir un warning específico por tipo
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


!pip install adjustText

In [2]:
# Enregistrer le temps de début
début = time.time()

In [3]:
#configuration
pd.set_option('display.max_row', 200)
pd.set_option('display.max_column', 70)
pd.set_option('display.precision', 5)
pd.set_option("max_colwidth", 80) 

In [4]:
# Versions
print('Version des librairies utilisées :')
print('Python          : ' + sys.version)
print('NumPy           : ' + np.version.full_version)
print('Pandas          : ' + pd.__version__)
print('Matplotlib      : ' + mpl.__version__)
print('Seaborn         : ' + sns.__version__)
print('Sklearn         : ' + sklearn.__version__)
print('Geopandas       : ' + sns.__version__)
print('IPython         : ' + ip.__version__)
print('statsmodels     : ' + stat.__version__)


now = datetime.datetime.now().isoformat()
print('Lancé le        : ' + now)

Version des librairies utilisées :
Python          : 3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]
NumPy           : 1.23.5
Pandas          : 1.5.3
Matplotlib      : 3.7.1
Seaborn         : 0.12.2
Sklearn         : 1.2.2
Geopandas       : 0.12.2
IPython         : 8.12.0
statsmodels     : 0.13.5
Lancé le        : 2023-06-30T09:00:37.316703


In [5]:
data = pd.read_csv('data_cleaned.csv')

In [6]:
data.shape

(93396, 30)

In [7]:
data.dropna(inplace=True)

In [8]:
data.shape

(93139, 30)

In [9]:
data.set_index('customer_unique_id', inplace=True)

In [10]:
data.head()

,Cat_la_plus_achetee,customer_zip_code_prefix,customer_city,customer_state,Nb_pdts,Tot_moy_achats,Delai_Moy_Commande,order_purchase_timestamp,heure_achat,jour_achat,delai_dernier_achat,Note_Moy_Com,Moy_Paiment,Facilités,geolocation_lat,geolocation_lng,home,appliances,construction,office,electronics,arts,fashion,health_beauty,sports_leisure,garden_pets,auto,food_drinks,other
customer_unique_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,home,7787,Other,SP,1.0,4.86676,7.0,2018-05-10 10:56:27,10.0,3.0,115.0,5.0,credit_card,8.0,-23.34026,-46.83108,129.9,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0000b849f77a49e4a4ce2b2a4ca5be3f,health_beauty,6053,osasco,SP,1.0,2.93916,4.0,2018-05-07 11:11:27,11.0,0.0,118.0,4.0,credit_card,1.0,-23.55904,-46.78852,0.0,0.0,0.0,0.0,0.00,0.0,0.0,18.9,0.0,0.0,0.0,0.0,0.0
0000f46a3911fa3c0805444483337064,office,88115,sao jose,SC,1.0,4.23411,26.0,2017-03-10 21:05:03,21.0,4.0,541.0,3.0,credit_card,8.0,-27.54301,-48.63494,0.0,0.0,0.0,69.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0000f6ccb0745a6a4b88665a16c9f078,electronics,66812,belem,PA,1.0,3.25771,21.0,2017-10-12 20:29:41,20.0,3.0,325.0,4.0,credit_card,4.0,-1.31273,-48.48275,0.0,0.0,0.0,0.0,25.99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0004aac84e0df4da2b147fca70cf8255,electronics,18040,sorocaba,SP,1.0,5.19296,14.0,2017-11-14 19:45:42,19.0,1.0,292.0,5.0,credit_card,6.0,-23.50559,-47.46982,0.0,0.0,0.0,0.0,180.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
data['order_purchase_timestamp'] = data['order_purchase_timestamp'].astype('datetime64')

In [12]:
data.drop(['customer_zip_code_prefix','Cat_la_plus_achetee'], axis=1, inplace=True)

In [13]:
data.drop(['customer_city', 'customer_state'], axis=1, inplace=True)

In [14]:
data.select_dtypes(['object']).nunique()

Moy_Paiment    4
dtype: int64

In [15]:
# Copie du dataframe
data_copie = data.copy()

<div class="alert alert-info">
    <font id="section_1">
        <h1 style="font-size:25px;"><strong>01. Ensemble du jeu de données</strong></h1>
</div>

### Préparation jeu de donnée complet

In [16]:
X_tot = data.copy()

categorical_columns = X_tot.select_dtypes(['category','object']).columns
numerical_columns   = X_tot.select_dtypes(['int32','float64']).columns

ohe = OneHotEncoder(sparse=False)
ss = StandardScaler()
X_tot[numerical_columns] = ss.fit_transform(X_tot[numerical_columns])
ohe.fit(X_tot[categorical_columns])

OneHotEncoder(sparse=False, sparse_output=False)

In [17]:
X_tot = pd.merge(X_tot[numerical_columns], pd.DataFrame(columns=ohe.get_feature_names_out().tolist(),
              data=ohe.fit_transform(X_tot[categorical_columns])).set_index(X_tot.index),
              left_index=True, right_index=True)

### ACP

In [18]:
pca_tot = PCA(random_state=1)

In [19]:
pca_tot.fit(X_tot)

PCA(random_state=1)

### Représentation des données

In [20]:
X_pca_tot = pca_tot.transform(X_tot)[:,:18]

In [21]:
colonnes_pca = X_tot.columns

## KMeans

Modèle KMeans avec le nombre de clusters optimal identifié sur le jeu de données réduit

In [22]:
#kmeans = KMeans(n_clusters=14, verbose=0, random_state=1).fit(X_pca_tot)
kmeans = KMeans(n_clusters=12, verbose=0, random_state=1).fit(X_pca_tot)

In [23]:
kmeans.labels_.shape

(93139,)

In [24]:
silhouette_score(X = X_pca_tot, labels= kmeans.labels_)

0.20476197085379394

In [25]:
inertia = {}
dict_kmeans = {}
dict_silhouette = {}
#for k in range(2, 12):
for k in range(2, 5):
    kmeans = KMeans(n_clusters=k,
                   verbose=0,
                   random_state=0).fit(X_pca_tot)
    inertia[k] = kmeans.inertia_
    dict_kmeans[k] = kmeans
    dict_silhouette[k] = silhouette_score(X = X_pca_tot, labels= kmeans.labels_)
    print('Boucle {} - Silhouette score : {}'.format(k, dict_silhouette[k]))
    

Boucle 2 - Silhouette score : 0.9873981549819149
Boucle 3 - Silhouette score : 0.21117613571131733
Boucle 4 - Silhouette score : 0.2100991279385895


<div class="alert alert-info">
    <font id="section_2">
        <h1 style="font-size:25px;"><strong>02. Simulation pour déterminer la fréquence nécessaire de mise à jour</strong></h1>
</div>

Préparation jeu de données

* Faisons un test de clustering en retirant les 3 derniers mois du jeu de données (test de faisabilité en retirant tous les clients qui ont fait un achat dans les 3 derniers mois)

In [26]:
# Copie du dataframe
data = data_copie.copy()

In [27]:
categorical_columns = data.select_dtypes(['category','object']).columns
numerical_columns   = data.select_dtypes(['int32','float64']).columns

In [28]:
data['order_purchase_timestamp'].describe()

count                   93139
unique                  92898
top       2018-07-28 13:11:22
freq                        3
first     2016-09-04 21:15:19
last      2018-09-03 09:06:57
Name: order_purchase_timestamp, dtype: object

<div class="alert alert-info">
    <font id="section_3">
        <h1 style="font-size:25px;"><strong>03. Modèle 1</strong></h1>
</div>

In [45]:
X_old_M10 = data[data['order_purchase_timestamp'] < '2017-10-04 09:04:45'].copy()

In [46]:
categorical_columns = X_old_M10.select_dtypes(['category','object']).columns
numerical_columns   = X_old_M10.select_dtypes(['int32','float64']).columns


X_old_M10[numerical_columns] = ss.transform(X_old_M10[numerical_columns])
ohe.fit(X_old_M10[categorical_columns])

OneHotEncoder(sparse=False, sparse_output=False)

In [47]:
X_old_M10 = pd.merge(X_old_M10[numerical_columns], 
          pd.DataFrame(columns = ohe.get_feature_names_out().tolist(),
              data = ohe.fit_transform(X_old_M10[categorical_columns])).set_index(X_old_M10.index),
        left_index = True, right_index = True)

### ACP M10

In [48]:
X_pca_M10 = pca_tot.transform(X_old_M10)[:,:18]

### K means M10

In [49]:
 kmeans_old_M10 = KMeans(n_clusters=k,
                   verbose=1,
                   random_state=0).fit(X_pca_M10)

Initialization complete
Iteration 0, inertia 237672.14180032606.
Iteration 1, inertia 209998.7574245036.
Iteration 2, inertia 205885.26256159626.
Iteration 3, inertia 204239.7385332379.
Iteration 4, inertia 203596.27380715433.
Iteration 5, inertia 203359.52866243478.
Iteration 6, inertia 203274.2271719912.
Iteration 7, inertia 203239.90641032267.
Iteration 8, inertia 203221.94896424148.
Iteration 9, inertia 203214.29723372654.
Iteration 10, inertia 203210.61184307706.
Iteration 11, inertia 203208.8469922852.
Iteration 12, inertia 203207.90507356956.
Converged at iteration 12: center shift 2.8267727267646962e-05 within tolerance 6.236969526691159e-05.
Initialization complete
Iteration 0, inertia 260249.05767558076.
Iteration 1, inertia 228720.282122558.
Iteration 2, inertia 225828.5812442737.
Iteration 3, inertia 224722.73315941403.
Iteration 4, inertia 224188.1343022975.
Iteration 5, inertia 223878.11215746988.
Iteration 6, inertia 223641.15597023233.
Iteration 7, inertia 223402.590822

<div class="alert alert-info">
    <font id="section_4">
        <h1 style="font-size:25px;"><strong>04. Modèle 2</strong></h1>
</div>

In [50]:
#Date à changer ??
X_old_M11 = data[data['order_purchase_timestamp'] < '2017-10-04 09:04:45'].copy()

In [51]:
categorical_columns = X_old_M11.select_dtypes(['category','object']).columns
numerical_columns = X_old_M11.select_dtypes(['int32','float64']).columns


X_old_M11[numerical_columns] = ss.transform(X_old_M11[numerical_columns])
ohe.fit(X_old_M11[categorical_columns])

OneHotEncoder(sparse=False, sparse_output=False)

In [52]:
X_old_M11 = pd.merge(X_old_M11[numerical_columns], 
          pd.DataFrame(columns = ohe.get_feature_names_out().tolist(),
              data = ohe.fit_transform(X_old_M11[categorical_columns])).set_index(X_old_M11.index),
        left_index = True, right_index = True)

### ACP M11

In [53]:
X_pca_M11 = pca_tot.transform(X_old_M11)[:,:18]

### K means M11

In [54]:
 kmeans_old_M11 = KMeans(n_clusters=k,
                   verbose=1,
                   random_state=0).fit(X_pca_M11)

Initialization complete
Iteration 0, inertia 237672.14180032606.
Iteration 1, inertia 209998.7574245036.
Iteration 2, inertia 205885.26256159626.
Iteration 3, inertia 204239.7385332379.
Iteration 4, inertia 203596.27380715433.
Iteration 5, inertia 203359.52866243478.
Iteration 6, inertia 203274.2271719912.
Iteration 7, inertia 203239.90641032267.
Iteration 8, inertia 203221.94896424148.
Iteration 9, inertia 203214.29723372654.
Iteration 10, inertia 203210.61184307706.
Iteration 11, inertia 203208.8469922852.
Iteration 12, inertia 203207.90507356956.
Converged at iteration 12: center shift 2.8267727267645048e-05 within tolerance 6.236969526691159e-05.
Initialization complete
Iteration 0, inertia 260249.05767558076.
Iteration 1, inertia 228720.282122558.
Iteration 2, inertia 225828.58124427372.
Iteration 3, inertia 224722.73315941403.
Iteration 4, inertia 224188.1343022975.
Iteration 5, inertia 223878.11215746988.
Iteration 6, inertia 223641.15597023233.
Iteration 7, inertia 223402.59082

Iteration 0, inertia 248800.58531213622.
Iteration 1, inertia 223306.48827349045.
Iteration 2, inertia 221102.64877138622.
Iteration 3, inertia 219909.9554198357.
Iteration 4, inertia 218678.37590489903.
Iteration 5, inertia 216708.98611147024.
Iteration 6, inertia 214168.13575783974.
Iteration 7, inertia 212503.92066647072.
Iteration 8, inertia 211975.12415172698.
Iteration 9, inertia 211803.6457466797.
Iteration 10, inertia 211742.1580833883.
Iteration 11, inertia 211721.48147511366.
Iteration 12, inertia 211714.67208847814.
Iteration 13, inertia 211712.7482769137.
Converged at iteration 13: center shift 6.183902096214165e-05 within tolerance 6.236969526691159e-05.


<div class="alert alert-info">
    <font id="section_5">
        <h1 style="font-size:25px;"><strong>05. Simulation</strong></h1>
</div>

In [57]:
data['order_purchase_timestamp'].describe()

count                   93139
unique                  92898
top       2018-07-28 13:11:22
freq                        3
first     2016-09-04 21:15:19
last      2018-09-03 09:06:57
Name: order_purchase_timestamp, dtype: object

Date d'achat minimale: 2016-09-04 21:15:19
Date d'achat maximale: 2018-10-17 17:30:18
Differencie en années: 2.115068493150685

In [74]:
from sklearn.metrics import adjusted_rand_score

def generate_client_file(data, time):
    # Esta función genera el archivo de clientes hasta la fecha 'time'
    # Se debe filtrar los datos hasta 'time' y crear las características necesarias
    # Retorna el archivo de clientes generado
    filtered_data = data[data['order_purchase_timestamp'] <= time]
    client_file = filtered_data[numerical_columns]
    return client_file


# Supongamos que los modelos iniciales estan en 'M0' y 'M1', y el archivo completo 'data' hasta la fecha de inicio 'T0'
#M0 = KMeans(n_clusters=k, random_state=0)
#M1 = KMeans(n_clusters=k, random_state=0)


M0 = kmeans_old_M10
M1 = kmeans_old_M11

# Definir la fecha de inicio T0 y el intervalo de tiempo entre simulaciones. Date d'achat minimale: 2016-09-04 21:15:19
T0 = pd.to_datetime('2016-09-04')
#interval = pd.DateOffset(months=24)
interval = pd.DateOffset(months=1)

# Crear listas para almacenar las fechas y los valores de ARI
dates = []
ARIs = []


#while T0 < pd.to_datetime('2020-10-17'):  # Ajustar la fecha de finalización  Fecha: 2018-08-04 00:00:00, ARI: -0.03400753674783449
#while T0 < pd.to_datetime('2018-10-17'):  # Ajustar la fecha de finalización según el archivo completo
#while T0 < pd.to_datetime('2018-09-03'):  # Ajustar la fecha de finalización según el archivo completo
while T0 < pd.to_datetime('2016-12-04'):  # Ajustar la fecha de finalización The best Fecha: 2016-10-04 00:00:00, ARI: 0.3119266055045872
    T1 = T0 + interval

    # Generar el archivo de clientes hasta la fecha T1 utilizando la función generate_client_file
    file_T1 = generate_client_file(data, T1)
    
    #
    print('file_T1 \n',file_T1.columns.sort_values().to_list()) 

    # Transformar el archivo de clientes utilizando el StandardScaler correspondiente a cada modelo
    file_T1_transformed_M0 = ss.transform(file_T1)
    file_T1_transformed_M1 = ss.transform(file_T1)

    #file_T1_transformed_M0
    #print('file_T1_transformed_M0',file_T1_transformed_M0.shape)
    print('file_T1_transformed_M0',file_T1_transformed_M0)
        
        
    # centroides de M0
    print('centroides',M0.cluster_centers_.shape)
    
    # Predecir los clusters utilizando el modelo inicial M0
    C1_init = M0.predict(file_T1_transformed_M0[:,:18])

    # Entrenar el nuevo modelo M1 utilizando el archivo de clientes hasta la fecha T1
    M1.fit(file_T1_transformed_M1)
    C1_new = M1.predict(file_T1_transformed_M1)

    # Calcular el valor de ARI entre los clusters predichos por M0 y M1
    ARI = adjusted_rand_score(C1_init, C1_new)

    # Almacenar la fecha y el valor de ARI
    dates.append(T1)
    ARIs.append(ARI)

    # Actualizar la fecha de inicio para la siguiente simulación
    T0 = T1

# Mostrar los resultados de la simulación
for date, ARI in zip(dates, ARIs):
    print(f"Fecha: {date}, ARI: {ARI}")


file_T1 
 ['Delai_Moy_Commande', 'Facilités', 'Nb_pdts', 'Note_Moy_Com', 'Tot_moy_achats', 'appliances', 'arts', 'auto', 'construction', 'delai_dernier_achat', 'electronics', 'fashion', 'food_drinks', 'garden_pets', 'geolocation_lat', 'geolocation_lng', 'health_beauty', 'heure_achat', 'home', 'jour_achat', 'office', 'other', 'sports_leisure']
file_T1_transformed_M0 [[-1.51136272e-01  1.67099822e-01 -6.70587505e+00  1.36252803e+00
   1.65874477e+00  2.99177550e+00 -2.34293792e+00 -7.11226811e-01
  -4.08805567e-01 -1.31804131e-01 -2.67727040e-02 -3.02910228e-02
  -1.26861582e-02 -1.26770780e-02 -7.17617692e-03 -2.44292305e-02
  -2.56623037e-02  1.67185147e-02 -9.50233673e-03 -3.13163628e-02
  -2.64123860e-02 -7.90120847e-03 -1.57852397e-02]
 [-1.51136272e-01 -1.43691786e-01  1.53978013e-01  1.36252803e+00
  -1.39291486e+00  2.98524690e+00 -2.34293792e+00  3.46242726e-02
   3.04122562e-01  8.89294458e-01 -1.56259610e-02 -3.02910228e-02
  -1.26861582e-02 -1.26770780e-02 -7.17617692e-03 -2.

Iteration 0, inertia 3348.0264173189444.
Iteration 1, inertia 2440.890818851409.
Iteration 2, inertia 2430.70475873808.
Iteration 3, inertia 2429.718901607382.
Converged at iteration 3: strict convergence.
Initialization complete
Iteration 0, inertia 3971.790667520803.
Iteration 1, inertia 2338.5457835874713.
Iteration 2, inertia 2280.879414859438.
Iteration 3, inertia 2252.374352985742.
Iteration 4, inertia 2240.620004145464.
Iteration 5, inertia 2237.6847066088735.
Iteration 6, inertia 2236.0742809976664.
Iteration 7, inertia 2235.5218792561504.
Converged at iteration 7: strict convergence.
Initialization complete
Iteration 0, inertia 2763.5584547114563.
Iteration 1, inertia 2419.3769245316726.
Iteration 2, inertia 2316.60448731056.
Iteration 3, inertia 2274.1797653922517.
Iteration 4, inertia 2250.2296104335733.
Iteration 5, inertia 2211.159112812753.
Iteration 6, inertia 2167.317573492369.
Iteration 7, inertia 2150.115380008307.
Iteration 8, inertia 2148.236872513629.
Iteration 9, 

pd.to_datetime('2016-12-04')                                                              
Fecha: 2018-09-04 00:00:00, ARI: -0.005336973901771135                                                             

pd.to_datetime('2018-09-03')                                                              
Fecha: 2018-09-04 00:00:00, ARI: -0.005336973901771135                                                             

pd.to_datetime('2018-10-17')                                                             
Fecha: 2018-09-04 00:00:00, ARI: -0.005336973901771135                                                             
Fecha: 2020-09-04 00:00:00, ARI: -0.005336973901771135                                                             

======================================================                                                             
#interval = pd.DateOffset(months=24)                                                             
**interval = pd.DateOffset(months=1)                                                             **
                                                             
Fecha: 2016-10-04 00:00:00, ARI: 0.3119266055045872                                                             
Fecha: 2016-11-04 00:00:00, ARI: 0.22627221191010818                                                             
Fecha: 2016-12-04 00:00:00, ARI: 0.22627221191010818                                                             
**Fecha: 2017-01-04 00:00:00, ARI: 0.22627221191010818**                                                           
Fecha: 2017-02-04 00:00:00, ARI: 0.15085426208521896                                                             
Fecha: 2017-03-04 00:00:00, ARI: 0.10657326844752564                                                             
Fecha: 2017-04-04 00:00:00, ARI: 0.0829995228127589                                                             
Fecha: 2017-05-04 00:00:00, ARI: 0.07505584438671475                                                             
Fecha: 2017-06-04 00:00:00, ARI: 0.06065334441764793                                                             
Fecha: 2017-07-04 00:00:00, ARI: 0.04818300576294785                                                             
Fecha: 2017-08-04 00:00:00, ARI: 0.03808701703661736                                                             
Fecha: 2017-09-04 00:00:00, ARI: 0.03246333394027523                                                             
Fecha: 2017-10-04 00:00:00, ARI: 0.004552822934703587                                                             
Fecha: 2017-11-04 00:00:00, ARI: -0.0029209948211162454                                                             
Fecha: 2017-12-04 00:00:00, ARI: 0.012595150400879296                                                             
Fecha: 2018-01-04 00:00:00, ARI: 0.007843509836757781                                                             
Fecha: 2018-02-04 00:00:00, ARI: -0.034057607463106744                                                             
Fecha: 2018-03-04 00:00:00, ARI: 0.0006342904370332947                                                             
                                                             
=======================================================                                                             
#interval = pd.DateOffset(months=24)                                                             
interval = pd.DateOffset(months=2)                                                             
                                                             
Fecha: 2016-11-04 00:00:00, ARI: 0.22627221191010818                                                             
Fecha: 2017-01-04 00:00:00, ARI: 0.22627221191010818                                                             
Fecha: 2017-03-04 00:00:00, ARI: 0.10657326844752564                                                             
Fecha: 2017-05-04 00:00:00, ARI: 0.07505584438671475                                                             
Fecha: 2017-07-04 00:00:00, ARI: 0.04818300576294785                                                             
Fecha: 2017-09-04 00:00:00, ARI: 0.03246333394027523                                                             
Fecha: 2017-11-04 00:00:00, ARI: -0.0029209948211162454                                                             
Fecha: 2018-01-04 00:00:00, ARI: 0.007843509836757781                                                             
Fecha: 2018-03-04 00:00:00, ARI: 0.0006342904370332947                                                             
Fecha: 2018-05-04 00:00:00, ARI: -0.03625197418843908                                                             
Fecha: 2018-07-04 00:00:00, ARI: 0.002963845267437217                                                             
Fecha: 2018-09-04 00:00:00, ARI: -0.005336973901771135                                                             
Fecha: 2018-11-04 00:00:00, ARI: -0.005336973901771135                                                             
                                                             
=======================================================                                                             
#interval = pd.DateOffset(months=24)                                                             
interval = pd.DateOffset(months=3)                                                             
Converged at iteration 30: center shift 8.618916376423436e-05 within tolerance 9.999999999998923e-05.                       
Fecha: 2016-12-04 00:00:00, ARI: 0.22627221191010818                                                             
Fecha: 2017-03-04 00:00:00, ARI: 0.10657326844752564                                                             
Fecha: 2017-06-04 00:00:00, ARI: 0.06065334441764793                                                             
Fecha: 2017-09-04 00:00:00, ARI: 0.03246333394027523                                                             
Fecha: 2017-12-04 00:00:00, ARI: 0.012595150400879296                                                             
Fecha: 2018-03-04 00:00:00, ARI: 0.0006342904370332947                                                             
Fecha: 2018-06-04 00:00:00, ARI: -0.03622546724302991                                                             
Fecha: 2018-09-04 00:00:00, ARI: -0.005336973901771135                                                             
Fecha: 2018-12-04 00:00:00, ARI: -0.005336973901771135                                                             
                                                             
=======================================================                                                             

#interval = pd.DateOffset(months=24)                                                             
interval = pd.DateOffset(months=4)                                                             
                                                             
Converged at iteration 30: center shift 8.618916376423077e-05 within tolerance 9.999999999998923e-05.                       
Fecha: 2017-01-04 00:00:00, ARI: 0.22627221191010818                                                             
Fecha: 2017-05-04 00:00:00, ARI: 0.07505584438671475                                                             
Fecha: 2017-09-04 00:00:00, ARI: 0.03246333394027523                                                             
Fecha: 2018-01-04 00:00:00, ARI: 0.007843509836757781                                                             
Fecha: 2018-05-04 00:00:00, ARI: -0.03625197418843908                                                             
Fecha: 2018-09-04 00:00:00, ARI: -0.005336973901771135                                                             
Fecha: 2019-01-04 00:00:00, ARI: -0.005336973901771135                                                             

<div class="alert alert-info">
    <font id="section_6">
        <h1 style="font-size:25px;"><strong>06. Conclusion</strong></h1>
</div>

En général, les valeurs de l'ARI restent relativement élevées, la plupart d'entre elles étant supérieures à 0. En général, une valeur d'ARI proche de 1 indique une similitude élevée entre les groupes de données, ce qui suggère que les données sont bien regroupées dans chaque intervalle d'un mois.

Cependant, il y a quelques fluctuations dans les valeurs de l'ARI au fil du temps. Certaines valeurs sont plus élevées, tandis que d'autres sont plus basses, ce qui indique qu'il peut y avoir des périodes où la similitude entre les groupes de données diminue.

In [40]:
#Enregistrer le temps de fin
fin = time.time()

In [41]:
#Calculer la durée en minutes
durée = (fin - début) / 60

In [42]:
#Afficher le temps d'exécution
print("Le temps d'exécution était de", durée, "minutes.")

Le temps d'exécution était de 7.304933710892995 minutes.
